In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-02-01"
end_date = "2019-03-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-02-01** to **2019-03-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 28 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                         repo  author
  46          nteract                          nes      16
  43          nteract                     hydrogen      10
  49          nteract                    scrapbook      10
  48          nteract                    papermill       7
  42          nteract                     examples       2
  45          nteract              meeting-minutes       2
  50          nteract                     semiotic       2
  44          nteract                          ion       1
  47          nteract                   nteract.io       1
  33       jupyterlab                   jupyterlab      38
  38       jupyterlab  jupyterlab-metadata-service      10
  34       jupyterlab        jupyterlab-commenting       5
  41       jupyterlab           theme-cookiecutter       5
  35       jupyterlab            jupyterlab-github       2
  37       jupyterlab             jupyterlab-latex       2
  39       jupyterlab               jupyterlab-toc       2
  32       jupyterlab            jupyter-renderers       1
  36       jupyterlab      jupyterlab-google-drive       1
  40       jupyterlab            jupyterlab_server       1
  23       jupyterhub                   jupyterhub      16
  26       jupyterhub          nativeauthenticator      15
  31       jupyterhub       zero-to-jupyterhub-k8s      15
  19       jupyterhub                    binderhub      10
  30       jupyterhub                traefik-proxy      10
  25       jupyterhub          mybinder.org-deploy       9
  22       jupyterhub         jupyter-server-proxy       6
  27       jupyterhub               oauthenticator       6
  21       jupyterhub                dockerspawner       4
  18       jupyterhub                       binder       3
  20       jupyterhub                   chartpress       3
  24       jupyterhub                  kubespawner       3
  29       jupyterhub      the-littlest-jupyterhub       3
  28       jupyterhub                 team-compass       2
  16  jupyter-widgets                   ipyleaflet      12
  17  jupyter-widgets                     tutorial       1
  15          jupyter                  repo2docker      14
  7           jupyter                 jupyter-book      11
  14          jupyter                     notebook       9
  6           jupyter           enterprise_gateway       5
  8           jupyter               jupyter_client       5
  5           jupyter                docker-stacks       4
  10          jupyter                        nb2kg       3
  11          jupyter                    nbconvert       3
  9           jupyter               kernel_gateway       1
  12          jupyter                       nbdime       1
  13          jupyter                     nbformat       1
  3           ipython                      ipython       4
  1           ipython                    ipykernel       1
  2           ipython                  ipyparallel       1
  4           ipython              ipython-website       1
  0   binder-examples             continuous-build       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nes', 'hydrogen', 'scrapbook', 'papermill', 'examples', 'meeting-minutes', 'semiotic', 'ion', 'nteract.io', 'jupyterlab', 'jupyterlab-metadata-service', 'jupyterlab-commenting', 'theme-cookiecutter', 'jupyterlab-github', 'jupyterlab-latex', 'jupyterlab-toc', 'jupyter-renderers', 'jupyterlab-google-drive', 'jupyterlab_server', 'jupyterhub', 'nativeauthenticator', 'zero-to-jupyterhub-k8s', 'binderhub', 'traefik-proxy', 'mybinder.org-deploy', 'jupyter-server-proxy', 'oauthenticator', 'dockerspawner', 'binder', 'chartpress', 'kubespawner', 'the-littlest-jupyterhub', 'team-compass', 'ipyleaflet', 'tutorial', 'repo2docker', 'jupyter-book', 'notebook', 'enterprise_gateway', 'jupyter_client', 'docker-stacks', 'nb2kg', 'nbconvert', 'kernel_gateway', 'nbdime', 'nbformat',

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:   username  authored authorAssociation
    0  davidar         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 28 days',
    width: 1000
  }), Chart({
    data:           username  authored authorAssociation
    0          Carreau         1            MEMBER
    1  Vlad-Shcherbina         1       CONTRIBUTOR
    2      flutefreak7         1              NONE
    3         memeplex         1       CONTRIBUTOR
    4            minrk         1            MEMBER
    5         ppmfloss         1       CONTRIBUTOR
    6          tomoboy         1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 28 days',
    width: 1000
  }), Chart({
    data:            username  authored authorAssociation
    8         choldgraf         7            MEMBER
    17      kevin-bates         6            MEMBER
    21            minrk         6            MEMBER
    7           betatim         5            MEMBER
    27        takluyver         3            MEMBER
    1    GladysNalvarte         2       CONTRIBUTOR
    12           esevan         2      COLLABORATOR
    30        yuvipanda         2            MEMBER
    28          travigd         2       CONTRIBUTOR
    25         steaward         1       CONTRIBUTOR
    18         lresende         1            MEMBER
    23         mrakitin         1       CONTRIBUTOR
    22           mpacer         1            MEMBER
    26         t-makaro         1      COLLABORATOR
    20       maxmouchet         1       CONTRIBUTOR
    29         twalcari         1       CONTRIBUTOR
    19  mathieuboudreau         1       CONTRIBUTOR
    24          parente         1            MEMBER
    0       EvgeniDubov         1       CONTRIBUTOR
    16       joergbrech         1       CONTRIBUTOR
    14         impact27         1       CONTRIBUTOR
    13    haraldschilly         1       CONTRIBUTOR
    11          emdupre         1      COLLABORATOR
    10         dylanzjy         1       CONTRIBUTOR
    9         dmikushin         1       CONTRIBUTOR
    6            arnavs         1       CONTRIBUTOR
    5       aayusharyan         1       CONTRIBUTOR
    4           Zsailer         1            MEMBER
    3             WrRan         1       CONTRIBUTOR
    2       SpencerPark         1       CONTRIBUTOR
    15        jakirkham         1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.827450

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org              repo username  closed
    0  binder-examples  continuous-build  betatim       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 28 days',
    width: 1000
  }), Chart({
    data:        org             repo username  closed
    1  ipython        ipykernel    minrk       1
    2  ipython      ipyparallel    minrk       1
    3  ipython          ipython  Carreau       4
    4  ipython  ipython-website  Carreau       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 28 days',
    width: 1000
  }), Chart({
    data:         org                repo     username  closed
    5   jupyter       docker-stacks      parente       4
    6   jupyter  enterprise_gateway  kevin-bates       5
    7   jupyter        jupyter-book    choldgraf       9
    8   jupyter        jupyter-book      emdupre       2
    9   jupyter      jupyter_client        minrk       4
    10  jupyter      jupyter_client       rgbkrk       1
    11  jupyter      kernel_gateway     rolweber       1
    12  jupyter               nb2kg  kevin-bates       3
    13  jupyter           nbconvert        MSeal       2
    14  jupyter           nbconvert     willingc       1
    15  jupyter              nbdime      vidartf       1
    16  jupyter            nbformat    takluyver       1
    17  jupyter            notebook        minrk       8
    18  jupyter            notebook    takluyver       1
    19  jupyter         repo2docker      betatim      11
    20  jupyter         repo2docker        minrk       2
    21  jupyter         repo2docker     willingc       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 28 days',
    width: 1000
  }), Chart({
    data:                 org        repo     username  closed
    22  jupyter-widgets  ipyleaflet  martinRenou      12
    23  jupyter-widgets    tutorial      mwcraig       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 28 days',
    width: 1000
  }), Chart({
    data:            org                     repo        username  closed
    24  jupyterhub                   binder         betatim       1
    25  jupyterhub                   binder       choldgraf       1
    26  jupyterhub                   binder          lheagy       1
    27  jupyterhub                binderhub         betatim       5
    28  jupyterhub                binderhub    captainsafia       1
    29  jupyterhub                binderhub       choldgraf       2
    30  jupyterhub                binderhub       yuvipanda       2
    31  jupyterhub               chartpress           minrk       3
    32  jupyterhub            dockerspawner           minrk       3
    33  jupyterhub            dockerspawner        willingc       1
    34  jupyterhub     jupyter-server-proxy      ryanlovett       2
    35  jupyterhub     jupyter-server-proxy       yuvipanda       4
    36  jupyterhub               jupyterhub         Carreau       2
    37  jupyterhub               jupyterhub         betatim       2
    38  jupyterhub               jupyterhub           minrk      11
    39  jupyterhub               jupyterhub        willingc       1
    40  jupyterhub              kubespawn

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0   binder-examples                 getting-data       1   
  1   binder-examples           minimal-dockerfile       1   
  2           ipython                  ipyparallel       1   
  3           ipython                      ipython      10   
  4           jupyter                         help      10   
  5           jupyter                      jupyter       5   
  6           jupyter                 jupyter-book       2   
  7           jupyter              jupyter_console       1   
  8           jupyter                       nbdime       2   
  9           jupyter                     notebook      35   
  10          jupyter                     steering       1   
  11  jupyter-widgets                   ipyleaflet       2   
  12  jupyter-widgets                   ipywidgets       5   
  13  jupyter-widgets                    pythreejs       1   
  14  jupyter-widgets                   traittypes       1   
  15  jupyter-widgets       widget-ts-cookiecutter       1   
  16       jupyterhub                    binderhub       3   
  17       jupyterhub       jupyter-rsession-proxy       1   
  18       jupyterhub         jupyter-server-proxy       2   
  19       jupyterhub                   jupyterhub       3   
  20       jupyterhub     jupyterhub-deploy-docker       2   
  21       jupyterhub               jupyterlab-hub       1   
  22       jupyterhub                  kubespawner       1   
  23       jupyterhub          mybinder.org-deploy       4   
  24       jupyterhub          nativeauthenticator       2   
  25       jupyterhub               oauthenticator       1   
  26       jupyterhub             tmpauthenticator       1   
  27       jupyterhub                  wrapspawner       1   
  28       jupyterhub       zero-to-jupyterhub-k8s       4   
  29       jupyterlab                   jupyterlab      16   
  30       jupyterlab               jupyterlab-git       2   
  31       jupyterlab            jupyterlab-github       1   
  32       jupyterlab      jupyterlab-google-drive       1   
  33       jupyterlab  jupyterlab-metadata-service       4   
  34       jupyterlab               jupyterlab-toc       2   
  35          nteract                    bookstore       3   
  36          nteract                           dx       1   
  37          nteract                      nteract       6   
  38          nteract                   nteract.io       3   
  39          nteract                    papermill       2   
  40          nteract                    scrapbook       6   
  41          nteract                     semiotic       4   
  42          nteract                  spawnteract       2   
  
                                    org/repo  
  0             binder-examples/getting-data  
  1       binder-examples/minimal-dockerfile  
  2                      ipython/ipyparallel  
  3                          ipython/ipython  
  4                             jupyter/help  
  5                          jupyter/jupyter  
  6                     jupyter/jupyter-book  
  7                  jupyter/jupyter_console  
  8                           jupyter/nbdime  
  9                         jupyter/notebook  
  10                        jupyter/steering  
  11              jupyter-widgets/ipyleaflet  
  12              jupyter-widgets/ipywidgets  
  13               jupyter-widgets/pythreejs  
  14              jupyter-widgets/traittypes  
  15  jupyter-widgets/widget-ts-cookiecutter  
  16                    jupyterhub/binderhub  
  17       jupyterhub/jupyter-rsession-proxy  
  18         jupyterhub/jupyter-server-proxy  
  19                   jupyterhub/jupyterhub  
  20     jupyterhub/jupyterhub-deploy-docker  
  21               jupyterhub/jupyterlab-hub  
  22                  jupyterhub/kubespawner  
  23          jupyterhub/mybinder.org-deploy  
  24          jupyterhub/nativeauthenticator  
  25               jupyterhub/oau

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0           ipython                    ipykernel       1   
  1           ipython                  ipyparallel       1   
  2           ipython                      ipython       3   
  3           jupyter                docker-stacks       3   
  4           jupyter           enterprise_gateway       8   
  5           jupyter                         help       1   
  6           jupyter                      jupyter       1   
  7           jupyter                 jupyter-book      12   
  8           jupyter               jupyter_client       1   
  9           jupyter               kernel_gateway       6   
  10          jupyter                    nbconvert       5   
  11          jupyter                       nbdime       2   
  12          jupyter                     nbformat       1   
  13          jupyter                     notebook       6   
  14          jupyter                  repo2docker       4   
  15  jupyter-widgets                   ipyleaflet       4   
  16  jupyter-widgets                   ipywidgets       4   
  17       jupyterhub                 batchspawner       1   
  18       jupyterhub                    binderhub       3   
  19       jupyterhub                   chartpress       2   
  20       jupyterhub                dockerspawner       1   
  21       jupyterhub         jupyter-server-proxy       2   
  22       jupyterhub                   jupyterhub      19   
  23       jupyterhub                  kubespawner       3   
  24       jupyterhub            ldapauthenticator       1   
  25       jupyterhub          mybinder.org-deploy       1   
  26       jupyterhub          nativeauthenticator       3   
  27       jupyterhub                  nbgitpuller       2   
  28       jupyterhub                 team-compass       8   
  29       jupyterhub      the-littlest-jupyterhub       2   
  30       jupyterhub                traefik-proxy       6   
  31       jupyterhub       zero-to-jupyterhub-k8s      10   
  32       jupyterlab                   jupyterlab      39   
  33       jupyterlab               jupyterlab-git       3   
  34       jupyterlab            jupyterlab-github       1   
  35       jupyterlab  jupyterlab-metadata-service       2   
  36       jupyterlab               jupyterlab-toc       1   
  37       jupyterlab           theme-cookiecutter       2   
  38          nteract                    bookstore       3   
  39          nteract                     examples       1   
  40          nteract                     hydrogen      26   
  41          nteract              meeting-minutes       1   
  42          nteract                    papermill       8   
  43          nteract                    scrapbook       3   
  44          nteract                     semiotic      10   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                      ipython/ipyparallel  
  2                          ipython/ipython  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                             jupyter/help  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                   jupyter/jupyter_client  
  9                   jupyter/kernel_gateway  
  10                       jupyter/nbconvert  
  11                          jupyter/nbdime  
  12                        jupyter/nbformat  
  13                        jupyter/notebook  
  14                     jupyter/repo2docker  
  15              jupyter-widgets/ipyleaflet  
  16              jupyter-widgets/ipywidgets  
  17                 jupyterhub/batchspawner  
  18                    jupyterhub/binderhub  
  19                   jupyterhub/chartpress  
  20                jupyterhub/dockerspawner  
  21         jupyterhub/jupyter-server-proxy  
  22                   jupyterhub/jupyterhub  
  2

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                         repo     kind  count
    29   jupyterlab                   jupyterlab  created   16.0
    30   jupyterlab               jupyterlab-git  created    2.0
    31   jupyterlab            jupyterlab-github  created    1.0
    32   jupyterlab      jupyterlab-google-drive  created    1.0
    33   jupyterlab  jupyterlab-metadata-service  created    4.0
    34   jupyterlab               jupyterlab-toc  created    2.0
    59   jupyterlab           theme-cookiecutter  created    0.0
    92   jupyterlab                   jupyterlab   closed   39.0
    93   jupyterlab               jupyterlab-git   closed    3.0
    94   jupyterlab            jupyterlab-github   closed    1.0
    95   jupyterlab      jupyterlab-google-drive   closed    0.0
    96   jupyterlab  jupyterlab-metadata-service   closed    2.0
    97   jupyterlab               jupyterlab-toc   closed    1.0
    122  jupyterlab           theme-cookiecutter   closed    2.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 28 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                 org                    repo     kind  count
    11  jupyter-widgets              ipyleaflet  created    2.0
    12  jupyter-widgets              ipywidgets  created    5.0
    13  jupyter-widgets               pythreejs  created    1.0
    14  jupyter-widgets              traittypes  created    1.0
    15  jupyter-widgets  widget-ts-cookiecutter  created    1.0
    74  jupyter-widgets              ipyleaflet   closed    4.0
    75  jupyter-widgets              ipywidgets   closed    4.0
    76  jupyter-widgets               pythreejs   closed    0.0
    77  jupyter-widgets              traittypes   closed    0.0
    78  jupyter-widgets  widget-ts-cookiecutter   closed    0.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 28 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org         repo     kind  count
    2    ipython  ipyparallel  created    1.0
    3    ipython      ipython  created   10.0
    43   ipython    ipykernel  created    0.0
    65   ipython  ipyparallel   closed    1.0
    66   ipython      ipython   closed    3.0
    106  ipython    ipykernel   closed    1.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 28 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
   

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                         repo   time_open  \
  0           ipython                    ipykernel    7.568333   
  1           ipython                  ipyparallel   11.165301   
  2           ipython                      ipython    1.902593   
  3           jupyter                docker-stacks    1.046447   
  4           jupyter           enterprise_gateway    8.655903   
  5           jupyter                         help    0.001782   
  6           jupyter                      jupyter    0.004850   
  7           jupyter                 jupyter-book   24.467315   
  8           jupyter               jupyter_client   93.085324   
  9           jupyter               kernel_gateway  412.736997   
  10          jupyter                    nbconvert   59.929306   
  11          jupyter                       nbdime    4.764259   
  12          jupyter                     nbformat    2.030197   
  13          jupyter                     notebook    8.435613   
  14          jupyter                  repo2docker   62.583200   
  15  jupyter-widgets                   ipyleaflet    3.607106   
  16  jupyter-widgets                   ipywidgets    0.128038   
  17       jupyterhub                 batchspawner    5.036655   
  18       jupyterhub                    binderhub    0.314433   
  19       jupyterhub                   chartpress  116.557072   
  20       jupyterhub                dockerspawner  880.865613   
  21       jupyterhub         jupyter-server-proxy   10.038281   
  22       jupyterhub                   jupyterhub    6.293009   
  23       jupyterhub                  kubespawner   36.807315   
  24       jupyterhub            ldapauthenticator   13.849132   
  25       jupyterhub          mybinder.org-deploy    2.923391   
  26       jupyterhub          nativeauthenticator    2.850926   
  27       jupyterhub                  nbgitpuller    3.640145   
  28       jupyterhub                 team-compass   55.272587   
  29       jupyterhub      the-littlest-jupyterhub    0.132946   
  30       jupyterhub                traefik-proxy   36.941082   
  31       jupyterhub       zero-to-jupyterhub-k8s    2.462292   
  32       jupyterlab                   jupyterlab   45.916910   
  33       jupyterlab               jupyterlab-git   39.041944   
  34       jupyterlab            jupyterlab-github   21.537465   
  35       jupyterlab  jupyterlab-metadata-service   24.574705   
  36       jupyterlab               jupyterlab-toc    0.027986   
  37       jupyterlab           theme-cookiecutter    3.282818   
  38          nteract                    bookstore    6.048241   
  39          nteract                     examples   29.635000   
  40          nteract                     hydrogen  298.636534   
  41          nteract              meeting-minutes    7.854676   
  42          nteract                    papermill    2.190289   
  43          nteract                    scrapbook    8.768148   
  44          nteract                     semiotic    3.479167   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                      ipython/ipyparallel  
  2                          ipython/ipython  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                             jupyter/help  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                   jupyter/jupyter_client  
  9                   jupyter/kernel_gateway  
  10                       jupyter/nbconvert  
  11                          jupyter/nbdime  
  12                        jupyter/nbformat  
  13                        jupyter/notebook  
  14                     jupyter/repo2docker  
  15              jupyter-widgets/ipyleaflet  
  16              jupyter-widgets/ipywidgets  
  17                 jupyterhub/batchspawner  
  18                    jupyterhub/binderhub  
  19   

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 28 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:     author  count authorAssociation
    0  betatim      2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 28 days',
    width: 1000
  }), Chart({
    data:                  author  count authorAssociation
    0               Carreau     61            MEMBER
    4        LucianaMarques     16       CONTRIBUTOR
    40                minrk      8            MEMBER
    54            tacaswell      5       CONTRIBUTOR
    37          meeseeksdev      4       CONTRIBUTOR
    38             memeplex      4       CONTRIBUTOR
    45             ppmfloss      4       CONTRIBUTOR
    12              albertz      4              NONE
    52               stas00      4              NONE
    8         SylvainCorlay      3            MEMBER
    28        happyconcepts      2              NONE
    55             tinkerjs      2              NONE
    25          flutefreak7      2              NONE
    34     joekingTheThird3      2              NONE
    43              njsmith      2       CONTRIBUTOR
    39   michaelsilverstein      1              NONE
    36             luckjiff      1              NONE
    41             mlazowik      1              NONE
    42          mrecachinas      1              NONE
    44                opi9a      1              NONE
    49             rmorshea      1              NONE
    46                qgbcs      1              NONE
    47          rainwoodman      1              NONE
    48             riow1983      1              NONE
    50              sintell      1              NONE
    51              slishak      1              NONE
    53              stmax82      1              NONE
    56               vityas      1              NONE
    57            westurner      1              NONE
    58               wjakob      1              NONE
    59             woodrujm      1              NONE
    60              xiaolai      1              NONE
    35           josemarfdc      1              NONE
    31             jdemeyer      1       CONTRIBUTOR
    33              jkbr-19      1              NONE
    32                jf---      1              NONE
    2                 FRidh      1              NONE
    3   KrishnaMohanRavilla      1              NONE
    5          PhysicsForce      1              NONE
    6              Pithikos      1              NONE
    7              Sapna714      1              NONE
    9      abduallahmohamed      1              NONE
    10            achampion      1              NONE
    11      akshayparopkari      1       CONTRIBUTOR
    13            altimmons      1              NONE
    14       andrii-rubtsov      1              NONE
    15             asmeurer      1       CONTRIBUTOR
    16         belgiancoder      1              NONE
    17           benlindsay      1              NONE
    18            benmezger      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:     author  n_first_responses authorAssociation
    0  betatim                  2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 28 days',
    width: 1000
  }), Chart({
    data:               author  n_first_responses authorAssociation
    0            Carreau                 25            MEMBER
    3     LucianaMarques                  4       CONTRIBUTOR
    27             minrk                  4            MEMBER
    31          ppmfloss                  3       CONTRIBUTOR
    26          memeplex                  2       CONTRIBUTOR
    37            stas00                  2              NONE
    33          riow1983                  1              NONE
    25          luckjiff                  1              NONE
    28          mlazowik                  1              NONE
    29       mrecachinas                  1              NONE
    30             opi9a                  1              NONE
    32             qgbcs                  1              NONE
    35           sintell                  1              NONE
    34          rmorshea                  1              NONE
    23           jkbr-19                  1              NONE
    36           slishak                  1              NONE
    38         tacaswell                  1       CONTRIBUTOR
    39          tinkerjs                  1              NONE
    40            vityas                  1              NONE
    41         westurner                  1              NONE
    42            wjakob                  1              NONE
    43           xiaolai                  1              NONE
    24        josemarfdc                  1              NONE
    22             jf---                  1              NONE
    1          EPedrotti                  1              NONE
    11      belgiancoder                  1              NONE
    2              FRidh                  1              NONE
    4           Pithikos                  1              NONE
    5           Sapna714                  1              NONE
    6   abduallahmohamed                  1              NONE
    7          achampion                  1              NONE
    8            albertz                  1              NONE
    9          altimmons                  1              NONE
    10    andrii-rubtsov                  1              NONE
    12        benlindsay                  1              NONE
    21         jcolonese                  1              NONE
    13      brettbrocato                  1              NONE
    14           cemdede                  1              NONE
    15            cmrfrd                  1              NONE
    16   drorcohengithub                  1              NONE
    17       flutefreak7                  1              NONE
    18       free-4-ever                  1              NONE
    19          gpotter2                  1       CONTRIBUTOR
    20        jankukacka                  1              NONE
    44            zakora                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          rang

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>